<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-youtube-transcription-utility/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert the YouTube Transaction Utility
## to run Google Colab in anticaption of using AI to:
* correct noun casing
* correct sentence ending - either a period, a question mark or a exclamation mark
* correct beginning of sentence casing

In [ ]:
!pip install openai python-dotenv google-generativeai anthropic gradio outube_transcript_api

In [ ]:
print("All good")

In [ ]:
# imports

from youtube_transcript_api import YouTubeTranscriptApi
# from flask import Flask, request, render_template
from app import utilities
# from config import Config

import os
import io
import sys
import json
import requests
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from google.colab import userdata

In [ ]:
# keys

openai_api_key = userdata.get("OPENAI_API_KEY")
claude_api_key = userdata.get("ANTHROPIC_API_KEY")
google_api_key = userdata.get("GOOGLE_API_KEY")
hugging_face_token = userdata.get("HF_TOKEN")

In [ ]:
# initialize

openai = OpenAI(api_key=openai_api_key)
claude = anthropic.Anthropic(api_key=claude_api_key)
google.generativeai.configure(api_key=google_api_key)

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-haiku-latest"
GOOGLE_MODEL = "gemini-2.5-flash-lite"

In [ ]:
system_message = "You are a Python code assistant. Your task is to analyze Python code and generate high-quality, concise comments and docstrings. Follow these guidelines:"
system_message += "Docstrings: Add a docstring for every function, class, and module. Describe the purpose of the function/class, its parameters, and its return value. Keep the description concise but informative, using proper Python docstring conventions (e.g., Google, NumPy, or reStructuredText format)."
system_message += "Inline Comments: Add inline comments only where necessary to clarify complex logic, important steps, or non-obvious behavior. Avoid commenting on obvious operations like x += 1 unless it involves a nuanced concept. Keep comments short, clear, and relevant."
system_message += "Typing: Strongly type all variables, arguments, classes, functions and modules."
system_message += "General Instructions: Maintain consistency in style and tone. Use technical terminology where appropriate, but ensure clarity for someone with intermediate Python knowledge. Do not over-explain or add redundant comments for self-explanatory code. Follow PEP 257 and PEP 8 standards for style and formatting."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Analyze the following Python code and enhance it by adding high-quality, concise docstrings and comments. "
    user_prompt += "Ensure all functions, classes, and modules have appropriate docstrings describing their purpose, parameters, and return values. "
    user_prompt += "Add inline comments only for complex or non-obvious parts of the code. "
    user_prompt += "Follow Python's PEP 257 and PEP 8 standards for documentation and formatting. "
    user_prompt += "Do not modify the code itself; only add annotations.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
def stream_gpt(python):
    print("stream_gpt")
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def stream_claude(python):
    print("stream_claude")
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
def stream_google(python):
    print("stream_google")
    # Initialize empty reply string
    reply = ""

    # The API for Gemini has a slightly different structure
    gemini = google.generativeai.GenerativeModel(
        model_name=GOOGLE_MODEL,
        system_instruction=system_message
    )

    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )

    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_google(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Comment and Type Provided Python Code")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        commented_python = gr.Textbox(label="Commented and Typed code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        comment = gr.Button("Comment and Type code")
    with gr.Row():
        python_run = gr.Button("Check Commented Python")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])

    comment.click(optimize, inputs=[python, model], outputs=[commented_python])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])

ui.launch(inbrowser=True, debug=True)